In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

import timeit

Using TensorFlow backend.


In [0]:
from google.colab import files
files.upload() 

Saving songs.csv to songs (1).csv


{'songs.csv': b'filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,flatness,poly,chroma_cqt,chroma_cens,spectral_contrast,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,tonnetz1,tonnetz2,tonnetz3,tonnetz4,tonnetz5,tonnetz6,label\r\nblues.00019.au,0.25725933452325717,0.0919802114367485,1195.4116396780198,1481.2842588449469,2234.878152664232,0.05887114817143963,0.017724959179759026,1.0862506804710352,0.5554187533122603,0.26817502402713733,21.991444648680176,-236.66534023125573,138.40905709839961,-4.903461529432489,47.62458245520507,5.179255781167525,8.879489246357975,0.507105477865866,-3.818273539489226,-14.977520288343268,10.42042596996878,-0.35118164431418697,0.7778293703530402,1.7773061497837663,4.235422613053339,-0.46335071496965186,-6.10569345003958,-5.3823819614052075,-1.3375862475512974,-3.719717449829062,-4.35315415746229,0.020574844317693283,0.01694321585337

In [0]:
ls

 sample_data/  'songs (1).csv'   songs.csv


In [0]:
data = pd.read_csv('songs.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,flatness,poly,chroma_cqt,...,mfcc18,mfcc19,mfcc20,tonnetz1,tonnetz2,tonnetz3,tonnetz4,tonnetz5,tonnetz6,label
0,blues.00019.au,0.257259,0.09198,1195.411640,1481.284259,2234.878153,0.058871,0.017725,1.086251,0.555419,...,-1.337586,-3.719717,-4.353154,0.020575,0.016943,0.088439,-0.026695,0.012183,-0.005494,blues
1,blues.00053.au,0.412501,0.09198,2171.221742,1954.383785,4237.132712,0.132802,0.000642,0.594293,0.513369,...,-0.271427,-3.696092,-0.504041,-0.002745,0.008571,-0.042509,0.057754,0.005468,0.017238,blues
2,blues.00048.au,0.379428,0.09198,2148.786199,2204.574064,4673.663406,0.098812,0.005946,1.750953,0.594625,...,0.464257,-1.909089,-3.128308,-0.010565,0.000952,0.003099,0.026811,-0.004230,0.006860,blues
3,blues.00001.au,0.340983,0.09198,1529.835316,2038.617579,3548.820207,0.056044,0.005735,1.223702,0.509443,...,0.293875,-0.287431,0.531573,-0.020591,0.000802,0.031767,-0.016651,0.005038,0.005216,blues
4,blues.00058.au,0.356588,0.09198,2068.371125,2033.930047,4231.499413,0.104372,0.002294,0.675315,0.503215,...,0.194597,0.842872,-3.416576,0.007437,0.018288,0.046516,0.011819,0.007859,-0.007299,blues


In [0]:
data = data.drop(['filename'],axis = 1)

In [0]:
data.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,flatness,poly,chroma_cqt,chroma_cens,...,mfcc18,mfcc19,mfcc20,tonnetz1,tonnetz2,tonnetz3,tonnetz4,tonnetz5,tonnetz6,label
0,0.257259,0.09198,1195.411640,1481.284259,2234.878153,0.058871,0.017725,1.086251,0.555419,0.268175,...,-1.337586,-3.719717,-4.353154,0.020575,0.016943,0.088439,-0.026695,0.012183,-0.005494,blues
1,0.412501,0.09198,2171.221742,1954.383785,4237.132712,0.132802,0.000642,0.594293,0.513369,0.267898,...,-0.271427,-3.696092,-0.504041,-0.002745,0.008571,-0.042509,0.057754,0.005468,0.017238,blues
2,0.379428,0.09198,2148.786199,2204.574064,4673.663406,0.098812,0.005946,1.750953,0.594625,0.274696,...,0.464257,-1.909089,-3.128308,-0.010565,0.000952,0.003099,0.026811,-0.004230,0.006860,blues
3,0.340983,0.09198,1529.835316,2038.617579,3548.820207,0.056044,0.005735,1.223702,0.509443,0.267173,...,0.293875,-0.287431,0.531573,-0.020591,0.000802,0.031767,-0.016651,0.005038,0.005216,blues
4,0.356588,0.09198,2068.371125,2033.930047,4231.499413,0.104372,0.002294,0.675315,0.503215,0.261366,...,0.194597,0.842872,-3.416576,0.007437,0.018288,0.046516,0.011819,0.007859,-0.007299,blues


In [0]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
X_train.shape

(800, 37)

In [0]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 256)               9728      
_________________________________________________________________
dense_18 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_19 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_20 (Dense)             (None, 10)                650       
Total params: 51,530
Trainable params: 51,530
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 1s 1ms/step - loss: 2.2139 - acc: 0.1575
Epoch 2/20
800/800 [==============================] - 0s 41us/step - loss: 1.8456 - acc: 0.3663
Epoch 3/20
800/800 [==============================] - 0s 39us/step - loss: 1.5755 - acc: 0.4337
Epoch 4/20
800/800 [==============================] - 0s 39us/step - loss: 1.3585 - acc: 0.5100
Epoch 5/20
800/800 [==============================] - 0s 39us/step - loss: 1.1981 - acc: 0.5775
Epoch 6/20
800/800 [==============================] - 0s 37us/step - loss: 1.0663 - acc: 0.6388
Epoch 7/20
800/800 [==============================] - 0s 37us/step - loss: 0.9643 - acc: 0.6675
Epoch 8/20
800/800 [==============================] - 0s 40us/step - loss: 0.8776 - acc: 0.7050
Epoch 9/20
800/800 [==============================] - 0s 42us/step - loss: 0.7895 - acc: 0.7338
Epoch 10/20
800/800 [==============================] - 0s 41us/step - loss: 0.7249 - acc:

In [0]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

200/200 [==============================] - 0s 351us/step
test_acc:  0.69
